### Imports

In [1]:
# Utils
from sklearn.model_selection import train_test_split
from imgaug import augmenters as iaa
import matplotlib.pyplot as plt
import imgaug as ia
import numpy as np
import cv2
import os

### Functions

In [2]:
def load_images_from_folder(folder, size = (256, 256), start = 0, end = 100000):
    images = []
    for filename in os.listdir(folder)[start:end]:
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            img = cv2.resize(img, dsize = size)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            images.append(img)
    return np.array(images)

In [3]:
def save_images_in_folder(folder, images, size = (256, 256), start_index = 0):
    for i, img in enumerate(images):
        img = cv2.resize(img, dsize = size)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        cv2.imwrite(os.path.join(folder, str(start_index + i) + '.png'), img)

In [4]:
def augment_images(folder, augmenter, images, size = (256, 256), start_index = 0, iterations = 10):
    n = len(images)
    for i in range(iterations):
        augmented_images=augmenter(images=images)
        save_images_in_folder(folder=folder, images=augmented_images, size=size, start_index = i*n)

### Augmenting dataset

In [5]:
ia.seed(1)

seq = iaa.Sequential([
    iaa.Flipud(0.5), # vertical flips
    iaa.Fliplr(0.5), # horizontal flips
    iaa.Crop(percent=(0, 0.1)), # random crops
    # Small gaussian blur with random sigma between 0 and 0.5.
    # But we only blur about 50% of all images.
    iaa.Sometimes(
        0.5,
        iaa.GaussianBlur(sigma=(0, 0.5))
    ),
    # Strengthen or weaken the contrast in each image.
    iaa.Sometimes(
        0.5,
        iaa.LinearContrast((0.75, 1.5))
    ),
    # Add gaussian noise.
    # For 50% of all images, we sample the noise once per pixel.
    # For the other 50% of all images, we sample the noise per pixel AND
    # channel. This can change the color (not only brightness) of the
    # pixels.
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255)),
    # Make some images brighter and some darker.
    # In 20% of all cases, we sample the multiplier once per channel,
    # which can end up changing the color of the images.
    iaa.Add((-50, 50)),
    iaa.Multiply((0.8, 1.2)),
    iaa.Sometimes(
        0.5,
        iaa.JpegCompression(compression=(70, 80)) # JPEG-compressing the images
    ),
    # Apply affine transformations to each image.
    # Scale/zoom them, translate/move them, rotate them and shear them.
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        rotate=(-25, 25),
        shear=(-8, 8)
    )], random_order=True) # apply augmenters in random order

In [6]:
dict_dir = {'organic': 'organic'}
dict_iter = {'organic': 9}
for key in dict_dir:
    imgs = load_images_from_folder(os.path.join(r'D:/Lucru/github-folder/hacktm-2022/dataset/', str(key)))
    augment_images(folder=os.path.join(r'D:/Lucru/github-folder/hacktm-2022/dataset-aug/', str(dict_dir[key])), augmenter=seq, images=imgs, iterations=dict_iter[key])